# TeleRetain: Phase 2 - Feature Engineering

This notebook demonstrates the advanced feature engineering pipeline for customer churn prediction.

In [5]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path
import os

try:
    current_dir = os.path.dirname(os.path.abspath(__file__))
    parent_dir = os.path.dirname(current_dir)
except NameError:
    parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from src.feature_engineering import AdvancedFeatureEngineer
from src.data.advanced_preprocessor import AdvancedPreprocessor, BusinessFeatureCreator

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

..\configs\model_config.yaml
..\configs\data_config.yaml
..\configs\deployment_config.yaml


## 1. Load and Explore Raw Data

In [ ]:
# Load the raw data
data_path = "../data/raw/telco_customer_churn.csv"
df_raw = pd.read_csv(data_path)

print(f"Raw Dataset Shape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")
print(f"\nData Types:")
print(df_raw.dtypes)

df_raw.head()

In [ ]:
# Check for missing values
missing_summary = df_raw.isnull().sum()
missing_summary = missing_summary[missing_summary > 0]

if len(missing_summary) > 0:
    print("Missing Values:")
    for col, count in missing_summary.items():
        percentage = (count / len(df_raw)) * 100
        print(f"  {col}: {count} ({percentage:.2f}%)")
else:
    print("No missing values found in the dataset")

## 2. Initialize Feature Engineering Pipeline

In [ ]:
# Initialize the advanced feature engineer
feature_engineer = AdvancedFeatureEngineer(output_dir="../data/processed/")

print("Feature Engineering Configuration:")
print(f"Target Column: {feature_engineer.config['target_column']}")
print(f"Positive Class: {feature_engineer.config['positive_class']}")
print(f"Business Features Enabled: {feature_engineer.config['feature_engineering']['business_features']}")
print(f"Interaction Features Enabled: {feature_engineer.config['feature_engineering']['interactions']['create']}")

## 3. Business Feature Creation

In [ ]:
# Create business features
business_creator = BusinessFeatureCreator(
    create_clv=True,
    create_service_metrics=True,
    create_risk_scores=True,
    create_customer_segments=True
)

df_with_business = business_creator.fit_transform(df_raw)
print(f"Shape after business features: {df_with_business.shape}")
print(f"Business features created: {len(business_creator.feature_names_)}")
print(f"New features: {business_creator.feature_names_}")

In [ ]:
# Visualize some key business features
if 'CLV_estimate' in df_with_business.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # CLV distribution by churn
    axes[0,0].hist([df_with_business[df_with_business['Churn']=='No']['CLV_estimate'].dropna(),
                   df_with_business[df_with_business['Churn']=='Yes']['CLV_estimate'].dropna()],
                  bins=30, alpha=0.7, label=['No Churn', 'Churn'])
    axes[0,0].set_title('CLV Distribution by Churn Status')
    axes[0,0].set_xlabel('Customer Lifetime Value')
    axes[0,0].legend()
    
    # Service adoption by churn
    if 'total_services' in df_with_business.columns:
        churn_by_services = df_with_business.groupby('total_services')['Churn'].apply(lambda x: (x=='Yes').mean() * 100)
        axes[0,1].bar(churn_by_services.index, churn_by_services.values, alpha=0.7, color='orange')
        axes[0,1].set_title('Churn Rate by Number of Services')
        axes[0,1].set_xlabel('Total Services')
        axes[0,1].set_ylabel('Churn Rate (%)')
    
    # Contract risk score distribution
    if 'contract_risk_score' in df_with_business.columns:
        axes[1,0].hist([df_with_business[df_with_business['Churn']=='No']['contract_risk_score'],
                       df_with_business[df_with_business['Churn']=='Yes']['contract_risk_score']],
                      bins=5, alpha=0.7, label=['No Churn', 'Churn'])
        axes[1,0].set_title('Contract Risk Score Distribution')
        axes[1,0].set_xlabel('Risk Score')
        axes[1,0].legend()
    
    # Family score impact
    if 'family_size_score' in df_with_business.columns:
        churn_by_family = df_with_business.groupby('family_size_score')['Churn'].apply(lambda x: (x=='Yes').mean() * 100)
        axes[1,1].bar(churn_by_family.index, churn_by_family.values, alpha=0.7, color='green')
        axes[1,1].set_title('Churn Rate by Family Score')
        axes[1,1].set_xlabel('Family Size Score')
        axes[1,1].set_ylabel('Churn Rate (%)')
    
    plt.tight_layout()
    plt.show()

## 4. Run Complete Feature Engineering Pipeline

In [ ]:
# Run the complete pipeline
report_path = feature_engineer.run_complete_pipeline(
    data_path=data_path,
    target_col='Churn'
)

print(f"✅ Feature engineering completed!")
print(f"📊 Report generated: {report_path}")

In [ ]:
# Load the processed data to examine results
processed_data_path = "../data/processed/train_processed.csv"
df_processed = pd.read_csv(processed_data_path)

print(f"Processed Dataset Shape: {df_processed.shape}")
print(f"Original Dataset Shape: {df_raw.shape}")
print(f"Features Added: {df_processed.shape[1] - df_raw.shape[1]}")

# Show new feature names
original_features = set(df_raw.columns)
new_features = [col for col in df_processed.columns if col not in original_features]
print(f"\nNew Features Created ({len(new_features)}):")
for i, feature in enumerate(new_features[:20], 1):  # Show first 20
    print(f"  {i:2d}. {feature}")
if len(new_features) > 20:
    print(f"  ... and {len(new_features) - 20} more")

## 5. Feature Analysis and Visualization

In [ ]:
# Analyze feature distributions
numeric_features = df_processed.select_dtypes(include=[np.number]).columns
categorical_features = df_processed.select_dtypes(include=['object', 'category']).columns

print(f"Feature Distribution Analysis:")
print(f"  Total Features: {len(df_processed.columns)}")
print(f"  Numeric Features: {len(numeric_features)}")
print(f"  Categorical Features: {len(categorical_features)}")

# Check for binary features
binary_features = [col for col in numeric_features 
                  if df_processed[col].nunique() == 2 and 
                     set(df_processed[col].unique()).issubset({0, 1})]
print(f"  Binary Features: {len(binary_features)}")

# Check data quality
missing_after = df_processed.isnull().sum().sum()
print(f"\nData Quality:")
print(f"  Missing values: {missing_after}")
print(f"  Duplicates: {df_processed.duplicated().sum()}")

In [ ]:
# Feature correlation analysis
if len(numeric_features) > 1:
    # Calculate correlations with target (if numeric)
    target_col = 'Churn'
    if target_col in df_processed.columns:
        # Convert target to numeric for correlation
        target_numeric = (df_processed[target_col] == 'Yes').astype(int) if df_processed[target_col].dtype == 'object' else df_processed[target_col]
        
        # Get correlations with target
        correlations = df_processed[numeric_features].corrwith(target_numeric).abs().sort_values(ascending=False)
        correlations = correlations.dropna()
        
        print(f"\nTop 15 Features by Correlation with {target_col}:")
        for i, (feature, corr) in enumerate(correlations.head(15).items(), 1):
            print(f"  {i:2d}. {feature}: {corr:.4f}")
        
        # Plot top correlations
        plt.figure(figsize=(12, 8))
        top_correlations = correlations.head(15)
        plt.barh(range(len(top_correlations)), top_correlations.values)
        plt.yticks(range(len(top_correlations)), top_correlations.index)
        plt.xlabel('Absolute Correlation with Churn')
        plt.title('Top 15 Features by Correlation with Churn')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()

In [ ]:
# Feature importance analysis using Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

if 'Churn' in df_processed.columns:
    # Prepare data for feature importance
    X = df_processed.drop(['Churn'], axis=1)
    y = (df_processed['Churn'] == 'Yes').astype(int)
    
    # Quick train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Train Random Forest for feature importance
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
    rf.fit(X_train, y_train)
    
    # Get feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 15 Features by Random Forest Importance:")
    for i, (_, row) in enumerate(feature_importance.head(15).iterrows(), 1):
        print(f"  {i:2d}. {row['feature']}: {row['importance']:.4f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    top_features = feature_importance.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Feature Importance')
    plt.title('Top 15 Features by Random Forest Importance')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    # Model performance on processed features
    from sklearn.metrics import classification_report, roc_auc_score
    
    y_pred = rf.predict(X_test)
    y_pred_proba = rf.predict_proba(X_test)[:, 1]
    
    print(f"\n🎯 Quick Model Performance on Engineered Features:")
    print(f"AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(f"\nClassification Report:")
    print(classification_report(y_test, y_pred))

## 6. Feature Engineering Impact Analysis

In [ ]:
# Compare model performance before and after feature engineering
print("📊 Comparing Model Performance: Before vs After Feature Engineering")
print("=" * 70)

# Prepare original data (basic preprocessing only)
df_basic = df_raw.copy()

# Basic preprocessing for original data
# Handle missing values in TotalCharges
df_basic['TotalCharges'] = pd.to_numeric(df_basic['TotalCharges'], errors='coerce')
df_basic['TotalCharges'].fillna(df_basic['MonthlyCharges'], inplace=True)

# One-hot encode categorical variables
categorical_columns = df_basic.select_dtypes(include=['object']).columns.tolist()
if 'Churn' in categorical_columns:
    categorical_columns.remove('Churn')
    
df_basic_encoded = pd.get_dummies(df_basic, columns=categorical_columns, drop_first=True)

# Prepare datasets
X_basic = df_basic_encoded.drop(['Churn'], axis=1)
X_engineered = df_processed.drop(['Churn'], axis=1)
y_basic = (df_basic['Churn'] == 'Yes').astype(int)
y_engineered = (df_processed['Churn'] == 'Yes').astype(int)

# Train-test splits
X_basic_train, X_basic_test, y_basic_train, y_basic_test = train_test_split(
    X_basic, y_basic, test_size=0.3, random_state=42, stratify=y_basic)

X_eng_train, X_eng_test, y_eng_train, y_eng_test = train_test_split(
    X_engineered, y_engineered, test_size=0.3, random_state=42, stratify=y_engineered)

# Train models
rf_basic = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_engineered = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

rf_basic.fit(X_basic_train, y_basic_train)
rf_engineered.fit(X_eng_train, y_eng_train)

# Predictions
y_basic_pred_proba = rf_basic.predict_proba(X_basic_test)[:, 1]
y_eng_pred_proba = rf_engineered.predict_proba(X_eng_test)[:, 1]

# Performance comparison
basic_auc = roc_auc_score(y_basic_test, y_basic_pred_proba)
engineered_auc = roc_auc_score(y_eng_test, y_eng_pred_proba)
improvement = engineered_auc - basic_auc

print(f"Basic Features (Original):     AUC = {basic_auc:.4f} | Features: {X_basic.shape[1]}")
print(f"Engineered Features:           AUC = {engineered_auc:.4f} | Features: {X_engineered.shape[1]}")
print(f"Improvement:                   +{improvement:.4f} ({improvement/basic_auc*100:.1f}% relative)")
print(f"\n🎉 Feature engineering {'improved' if improvement > 0 else 'did not improve'} model performance!")

## 7. Summary and Next Steps

In [ ]:
print("🎯 PHASE 2 FEATURE ENGINEERING SUMMARY")
print("=" * 50)
print(f"✅ Original Features: {df_raw.shape[1]}")
print(f"✅ Final Features: {df_processed.shape[1]}")
print(f"✅ Features Added: {df_processed.shape[1] - df_raw.shape[1]}")
print(f"✅ Data Quality: {df_processed.isnull().sum().sum()} missing values (vs {df_raw.isnull().sum().sum()} originally)")
print(f"✅ Model Performance: AUC improved by {improvement:.4f}")
print(f"\n📁 Outputs Generated:")
print(f"  • Processed Data: ../data/processed/train_processed.csv")
print(f"  • Feature Report: ../data/processed/feature_engineering_report.json")
print(f"  • Preprocessing Pipeline: ../data/processed/preprocessing_artifacts.pkl")
print(f"  • HTML Report: {report_path}")
print(f"\n🚀 Ready for Phase 3: Model Development & Training!")